## Github Analysis notebook
This notebook contains scripts to process github experiment results and plot them out (for single-threaded run)

In [72]:
import pandas as pd
import json
import zipfile

In [73]:
experimental_zip_path = '../results-general-dict.zip'

In [75]:
# load logic + pre-processing
def extract_per_file_stats_tuplex(job_stats):
        n_requests = len(job_stats['responses'])
        L =  [job_stats['responses'][i]['stats'] for i in range(n_requests)]
        for i in range(n_requests):
            req_uri = job_stats['responses'][i]['request'][0]
            L[i]['input_path'] = req_uri[:req_uri.rfind(':')]
            L[i]['req_uri'] = req_uri
            L[i]['duration'] = L[i]['request_total_time']
            L[i]['num_input_rows'] = L[i]['input']['total_input_row_count']
            L[i]['num_output_rows'] = L[i]['output']['normal']
        return L

def load_tuplex_path(fp, path):
    lines = fp.readlines()
    rows = [json.loads(line) for line in lines]

    data = []
    for row in rows:
        if 'python-baseline' not in path:
            ans = {'benchmark': 'github'}
            for k in ['benchmark', 'input_path', 'job_time_in_s', 'metrics', 'mode', 'options', 'output_path', 'scratch_path', 'startup_time_in_s']:
                ans[k] = row.get(k)
            try:
                ans['per_file_stats'] = extract_per_file_stats_tuplex(row['detailed_job_stats'])
            except:
                print(f'--- ERR: extract failed for path {path}')
            ans['llvm'] = 16 if 'llvm16' in path else 9
            row = ans
        data.append(row)
    return pd.DataFrame(data)

def load_tuplex_runs(zf):
    df_tplx = pd.DataFrame()
    for path in filter(lambda name: name.endswith('.ndjson'), zf.namelist()):
        print(f'loading {path}')
        df_tplx = pd.concat((df_tplx, load_tuplex_path(zf.open(path), path)))
    return df_tplx
        
def load_cc_baselines(zf):
    df_cc = pd.DataFrame()
    # the C++ baseline are directly stored as csv
    for path in filter(lambda name: name.endswith('.csv'), zf.namelist()):
        print(f'loading {path}')
        df_cc = pd.concat((df_cc, pd.read_csv(zf.open(path))))
    df_cc = df_cc[df_cc['mode'] != 'mode']

    for name in ['time_in_s', 'loading_time_in_s', 'total_time_in_s']:
        df_cc[name] = df_cc[name].astype(float)
    for name in ['input_row_count', 'output_row_count']:
        df_cc[name] = df_cc[name].astype(int)
    return df_cc

with zipfile.ZipFile(experimental_zip_path, 'r') as zf:
    # print(zf.namelist())
    df_cc = load_cc_baselines(zf)
   
    df_tplx = load_tuplex_runs(zf)
    
#     with myzip.open('document.txt') as myfile:
#         print(myfile.read())

loading results-general-dict/local-exp-llvm16/c++-baseline/github/best_results.csv
loading results-general-dict/local-exp-llvm16/c++-baseline/github/cjson_results.csv
loading results-general-dict/local-exp-llvm16/c++-baseline/github/yyjson_results.csv
loading results-general-dict/local-exp-llvm16/c++-baseline/github/cstruct_results.csv
loading results-general-dict/local-exp-llvm9/tuplex/github/hyper/results.ndjson
loading results-general-dict/local-exp-llvm9/tuplex/github/hyper-noopt/results.ndjson
loading results-general-dict/local-exp-llvm9/tuplex/github/nohyper/results.ndjson
loading results-general-dict/local-exp-llvm9/python-baseline/github/results.ndjson
loading results-general-dict/local-exp-llvm16/tuplex/github/hyper/results.ndjson
loading results-general-dict/local-exp-llvm16/tuplex/github/hyper-noopt/results.ndjson
loading results-general-dict/local-exp-llvm16/tuplex/github/nohyper/results.ndjson
loading results-general-dict/local-exp-llvm16/python-baseline/github/results.ndj

In [76]:
df_cc

,mode,input_path,output_path,time_in_s,loading_time_in_s,total_time_in_s,input_row_count,output_row_count
0,best,/hot/data/github_daily/2011-10-15.json,./local-exp/c++-baseline/github/best/output/pa...,0.060073,0.020326,21.773260,48899,1418
1,best,/hot/data/github_daily/2012-10-15.json,./local-exp/c++-baseline/github/best/output/pa...,0.193989,0.051573,21.773260,165692,5907
2,best,/hot/data/github_daily/2013-10-15.json,./local-exp/c++-baseline/github/best/output/pa...,0.411413,0.130098,21.773260,296456,10628
3,best,/hot/data/github_daily/2014-10-15.json,./local-exp/c++-baseline/github/best/output/pa...,1.542250,0.621012,21.773260,476393,16014
4,best,/hot/data/github_daily/2015-10-15.json,./local-exp/c++-baseline/github/best/output/pa...,1.385090,0.521614,21.773260,737714,25301
...,...,...,...,...,...,...,...,...
66,cstruct,/hot/data/github_daily/2017-10-15.json,./local-exp/c++-baseline/github/cstruct/output...,1.802470,0.529669,25.819719,910100,26107
67,cstruct,/hot/data/github_daily/2018-10-15.json,./local-exp/c++-baseline/github/cstruct/output...,3.494740,1.091120,25.819719,1522655,44404
68,cstruct,/hot/data/github_daily/2019-10-15.json,./local-exp/c++-baseline/github/cstruct/output...,5.122230,1.661090,25.819719,2134789,61337
69,cstruct,/hot/data/github_daily/2020-10-15.json,./local-exp/c++-baseline/github/cstruct/output...,7.403400,2.612160,25.819719,2963694,62354


In [77]:
df_tplx = df_tplx.reset_index().rename(columns={'index':'run', 'job_time_in_s':'total_time_in_s'})
df_tplx['run'] = df_tplx['run'] + 1
df_tplx.head()

,run,benchmark,input_path,total_time_in_s,metrics,mode,options,output_path,scratch_path,startup_time_in_s,per_file_stats,llvm,total_input_paths_size_in_bytes,total_input_rows,total_output_rows
0,1,github,/hot/data/github_daily/*.json,171.377957,"{'generate_llvm_time_s': 0, 'llvm_compilation_...",tuplex,"{'tuplex.allowUndefinedBehavior': False, 'tupl...",./local-exp/tuplex/github/hyper/output,./local-exp/scratch,0.019958,"[{'hyper_active': True, 'input': {'fallback': ...",9.0,NaN,NaN,NaN
1,2,github,/hot/data/github_daily/*.json,169.666286,"{'generate_llvm_time_s': 0, 'llvm_compilation_...",tuplex,"{'tuplex.allowUndefinedBehavior': False, 'tupl...",./local-exp/tuplex/github/hyper/output,./local-exp/scratch,0.020359,"[{'hyper_active': True, 'input': {'fallback': ...",9.0,NaN,NaN,NaN
2,3,github,/hot/data/github_daily/*.json,170.050273,"{'generate_llvm_time_s': 0, 'llvm_compilation_...",tuplex,"{'tuplex.allowUndefinedBehavior': False, 'tupl...",./local-exp/tuplex/github/hyper/output,./local-exp/scratch,0.020020,"[{'hyper_active': True, 'input': {'fallback': ...",9.0,NaN,NaN,NaN
3,4,github,/hot/data/github_daily/*.json,171.278456,"{'generate_llvm_time_s': 0, 'llvm_compilation_...",tuplex,"{'tuplex.allowUndefinedBehavior': False, 'tupl...",./local-exp/tuplex/github/hyper/output,./local-exp/scratch,0.020803,"[{'hyper_active': True, 'input': {'fallback': ...",9.0,NaN,NaN,NaN
4,1,github,/hot/data/github_daily/*.json,170.569332,"{'generate_llvm_time_s': 0, 'llvm_compilation_...",tuplex,"{'tuplex.allowUndefinedBehavior': False, 'tupl...",./local-exp/tuplex/github/hyper-noopt/output,./local-exp/scratch,0.020239,"[{'hyper_active': True, 'input': {'fallback': ...",9.0,NaN,NaN,NaN


In [78]:
# expand per_file_stats in df
def preprocess_tplx_df(df_tplx):
    df = df_tplx.copy()
    #df = df.dropna()
    df.drop(columns='input_path', inplace=True)
    columns_to_keep = ['run', 'benchmark', 'input_path',
                           'total_time_in_s', 'mode', 'output_path', 'startup_time_in_s']
    df = df.explode('per_file_stats')
    df.reset_index(drop=True, inplace=True)
    
    def expand_helper(t):
        if pd.isna(t) or t is None:
            return pd.Series()
        # example looks like this
        # {'hyper_active': True,
        #   'input': {'fallback': 0,
        #    'general': 0,
        #    'input_file_count': 1,
        #    'normal': 48899,
        #    'total_input_row_count': 48899,
        #    'unresolved': 0},
        #   'output': {'except': 0, 'normal': 1418},
        #   'request_total_time': 0.743892436,
        #   'spills': {'count': 0, 'size': 0},
        #   'timings': {'compile_time': 0.0298965,
        #    'fast_path_execution_time': 0.413112,
        #    'general_and_interpreter_time': 4.3911e-05,
        #    'hyperspecialization_time': 0.279614},
        #   'input_path': '/hot/data/github_daily/2011-10-15.json',
        #   'req_uri': '/hot/data/github_daily/2011-10-15.json:0-78478920',
        #   'duration': 0.743892436,
        #   'num_input_rows': 48899,
        #   'num_output_rows': 1418}
        data = {'input_row_count': t['num_input_rows'],
                'output_row_count': t['num_output_rows'],
                'time_in_s': t['duration'],
                'input_path': t['input_path']
               }
        if t.get('hyper_active') is not None:
            more_data = {'hyper': t.get('hyper_active'),
                    'request_time_in_s': t['request_total_time'],
                    'compile_time_in_s': t['timings']['compile_time'],
                    'fast_path_execution_time_in_s':t['timings']['fast_path_execution_time'],
                    'general_and_interpreter_time_in_s':t['timings']['general_and_interpreter_time'],
                    'hyperspecialization_time':t['timings'].get('hyperspecialization_time'),}
            data.update(more_data)
        
        return pd.Series(list(data.values()), index=(data.keys()))
    df = pd.merge(df, df['per_file_stats'].apply(expand_helper), left_index=True, right_index=True)
    df.drop(columns=['options', 'scratch_path'], inplace=True)
    
    def adjust_mode(row):
        mode = row['mode']
        op = row['output_path']
        llvm = row['llvm']
        if './local-exp/tuplex/github/' in op:
            return f'tuplex-llvm{int(llvm)}-' + op.replace('./local-exp/tuplex/github/', '').replace('/output', '')
        elif 'python-baseline' in op:
            return 'python'
        else:
            return mode
    df['mode'] = df[['mode', 'output_path', 'llvm']].apply(adjust_mode, axis=1)
    
    return df

In [79]:
df = preprocess_tplx_df(df_tplx)

In [80]:
df.head()

,run,benchmark,total_time_in_s,metrics,mode,output_path,startup_time_in_s,per_file_stats,llvm,total_input_paths_size_in_bytes,...,input_row_count,output_row_count,time_in_s,input_path,hyper,request_time_in_s,compile_time_in_s,fast_path_execution_time_in_s,general_and_interpreter_time_in_s,hyperspecialization_time
0,1,github,171.377957,"{'generate_llvm_time_s': 0, 'llvm_compilation_...",tuplex-llvm9-hyper,./local-exp/tuplex/github/hyper/output,0.019958,"{'hyper_active': True, 'input': {'fallback': 0...",9.0,NaN,...,48899,1418,0.743892,/hot/data/github_daily/2011-10-15.json,True,0.743892,0.029896,0.413112,0.000044,0.279614
1,1,github,171.377957,"{'generate_llvm_time_s': 0, 'llvm_compilation_...",tuplex-llvm9-hyper,./local-exp/tuplex/github/hyper/output,0.019958,"{'hyper_active': True, 'input': {'fallback': 8...",9.0,NaN,...,165692,5907,1.792307,/hot/data/github_daily/2012-10-15.json,True,1.792307,0.022970,1.351070,0.127788,0.272553
2,1,github,171.377957,"{'generate_llvm_time_s': 0, 'llvm_compilation_...",tuplex-llvm9-hyper,./local-exp/tuplex/github/hyper/output,0.019958,"{'hyper_active': True, 'input': {'fallback': 1...",9.0,NaN,...,296456,10628,3.366036,/hot/data/github_daily/2013-10-15.json,True,3.366036,0.023286,2.882480,0.192423,0.249389
3,1,github,171.377957,"{'generate_llvm_time_s': 0, 'llvm_compilation_...",tuplex-llvm9-hyper,./local-exp/tuplex/github/hyper/output,0.019958,"{'hyper_active': True, 'input': {'fallback': 1...",9.0,NaN,...,476393,16014,10.867659,/hot/data/github_daily/2014-10-15.json,True,10.867659,0.020366,10.561600,0.037634,0.229411
4,1,github,171.377957,"{'generate_llvm_time_s': 0, 'llvm_compilation_...",tuplex-llvm9-hyper,./local-exp/tuplex/github/hyper/output,0.019958,"{'hyper_active': True, 'input': {'fallback': 0...",9.0,NaN,...,737714,25301,9.942523,/hot/data/github_daily/2015-10-15.json,True,9.942523,0.029784,9.623150,0.000050,0.270989


In [81]:
# merge cc and tuplex together
df = pd.concat((df, df_cc))



In [82]:
df

/usr/local/lib/python3.10/site-packages/ipython-8.5.0-py3.10.egg/IPython/lib/pretty.py:778: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  output = repr(obj)
/usr/local/lib/python3.10/site-packages/ipython-8.5.0-py3.10.egg/IPython/core/formatters.py:343: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  return method()


,run,benchmark,total_time_in_s,metrics,mode,output_path,startup_time_in_s,per_file_stats,llvm,total_input_paths_size_in_bytes,...,output_row_count,time_in_s,input_path,hyper,request_time_in_s,compile_time_in_s,fast_path_execution_time_in_s,general_and_interpreter_time_in_s,hyperspecialization_time,loading_time_in_s
0,1.0,github,171.377957,"{'generate_llvm_time_s': 0, 'llvm_compilation_...",tuplex-llvm9-hyper,./local-exp/tuplex/github/hyper/output,0.019958,"{'hyper_active': True, 'input': {'fallback': 0...",9.0,NaN,...,1418,0.743892,/hot/data/github_daily/2011-10-15.json,True,0.743892,0.029896,0.413112,0.000044,0.279614,NaN
1,1.0,github,171.377957,"{'generate_llvm_time_s': 0, 'llvm_compilation_...",tuplex-llvm9-hyper,./local-exp/tuplex/github/hyper/output,0.019958,"{'hyper_active': True, 'input': {'fallback': 8...",9.0,NaN,...,5907,1.792307,/hot/data/github_daily/2012-10-15.json,True,1.792307,0.022970,1.351070,0.127788,0.272553,NaN
2,1.0,github,171.377957,"{'generate_llvm_time_s': 0, 'llvm_compilation_...",tuplex-llvm9-hyper,./local-exp/tuplex/github/hyper/output,0.019958,"{'hyper_active': True, 'input': {'fallback': 1...",9.0,NaN,...,10628,3.366036,/hot/data/github_daily/2013-10-15.json,True,3.366036,0.023286,2.882480,0.192423,0.249389,NaN
3,1.0,github,171.377957,"{'generate_llvm_time_s': 0, 'llvm_compilation_...",tuplex-llvm9-hyper,./local-exp/tuplex/github/hyper/output,0.019958,"{'hyper_active': True, 'input': {'fallback': 1...",9.0,NaN,...,16014,10.867659,/hot/data/github_daily/2014-10-15.json,True,10.867659,0.020366,10.561600,0.037634,0.229411,NaN
4,1.0,github,171.377957,"{'generate_llvm_time_s': 0, 'llvm_compilation_...",tuplex-llvm9-hyper,./local-exp/tuplex/github/hyper/output,0.019958,"{'hyper_active': True, 'input': {'fallback': 0...",9.0,NaN,...,25301,9.942523,/hot/data/github_daily/2015-10-15.json,True,9.942523,0.029784,9.623150,0.000050,0.270989,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,NaN,NaN,25.819719,NaN,cstruct,./local-exp/c++-baseline/github/cstruct/output...,NaN,NaN,NaN,NaN,...,26107,1.802470,/hot/data/github_daily/2017-10-15.json,NaN,NaN,NaN,NaN,NaN,NaN,0.529669
67,NaN,NaN,25.819719,NaN,cstruct,./local-exp/c++-baseline/github/cstruct/output...,NaN,NaN,NaN,NaN,...,44404,3.494740,/hot/data/github_daily/2018-10-15.json,NaN,NaN,NaN,NaN,NaN,NaN,1.091120
68,NaN,NaN,25.819719,NaN,cstruct,./local-exp/c++-baseline/github/cstruct/output...,NaN,NaN,NaN,NaN,...,61337,5.122230,/hot/data/github_daily/2019-10-15.json,NaN,NaN,NaN,NaN,NaN,NaN,1.661090
69,NaN,NaN,25.819719,NaN,cstruct,./local-exp/c++-baseline/github/cstruct/output...,NaN,NaN,NaN,NaN,...,62354,7.403400,/hot/data/github_daily/2020-10-15.json,NaN,NaN,NaN,NaN,NaN,NaN,2.612160


In [83]:
df.groupby(['mode', 'input_path']).mean().reset_index().sort_values(by=['input_path', 'time_in_s']).tail(10)[['input_path', 'mode', 'time_in_s', 'fast_path_execution_time_in_s']]

/var/folders/l7/8zgzcszx7z5gk7kk92f6nc1c0000gn/T/ipykernel_9164/4007452804.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby(['mode', 'input_path']).mean().reset_index().sort_values(by=['input_path', 'time_in_s']).tail(10)[['input_path', 'mode', 'time_in_s', 'fast_path_execution_time_in_s']]


,input_path,mode,time_in_s,fast_path_execution_time_in_s
32,/hot/data/github_daily/2021-10-15.json,cstruct,2.636850,NaN
120,/hot/data/github_daily/2021-10-15.json,yyjson,7.037760,NaN
43,/hot/data/github_daily/2021-10-15.json,python,13.049705,NaN
21,/hot/data/github_daily/2021-10-15.json,cjson,14.711950,NaN
98,/hot/data/github_daily/2021-10-15.json,tuplex-llvm9-hyper-noopt,16.104950,15.832725
65,/hot/data/github_daily/2021-10-15.json,tuplex-llvm16-hyper-noopt,16.116117,15.842150
109,/hot/data/github_daily/2021-10-15.json,tuplex-llvm9-nohyper,16.131762,16.090250
76,/hot/data/github_daily/2021-10-15.json,tuplex-llvm16-nohyper,16.191850,16.148550
54,/hot/data/github_daily/2021-10-15.json,tuplex-llvm16-hyper,16.318176,15.897350
87,/hot/data/github_daily/2021-10-15.json,tuplex-llvm9-hyper,16.392681,15.979450


In [84]:
df['mode'].unique()

array(['tuplex-llvm9-hyper', 'tuplex-llvm9-hyper-noopt',
       'tuplex-llvm9-nohyper', 'python', 'tuplex-llvm16-hyper',
       'tuplex-llvm16-hyper-noopt', 'tuplex-llvm16-nohyper', 'best',
       'cjson', 'yyjson', 'cstruct'], dtype=object)

In [85]:
df.groupby('mode').mean()

/var/folders/l7/8zgzcszx7z5gk7kk92f6nc1c0000gn/T/ipykernel_9164/1219598166.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby('mode').mean()


,run,total_time_in_s,startup_time_in_s,llvm,total_input_paths_size_in_bytes,total_input_rows,total_output_rows,input_row_count,output_row_count,time_in_s,request_time_in_s,compile_time_in_s,fast_path_execution_time_in_s,general_and_interpreter_time_in_s,hyperspecialization_time,loading_time_in_s
mode,,,,,,,,,,,,,,,,
best,NaN,22.072186,NaN,NaN,NaN,NaN,NaN,1.001151e+06,26745.0,2.006413,NaN,NaN,NaN,NaN,NaN,0.771506
cjson,NaN,155.714987,NaN,NaN,NaN,NaN,NaN,1.001151e+06,26745.0,14.155885,NaN,NaN,NaN,NaN,NaN,0.774448
cstruct,NaN,25.833117,NaN,NaN,NaN,NaN,NaN,1.001151e+06,26745.0,2.348444,NaN,NaN,NaN,NaN,NaN,0.775874
python,2.777778,137.252742,0.000000,NaN,3.807602e+10,11012665.0,294195.0,1.001151e+06,26745.0,12.452242,NaN,NaN,NaN,NaN,NaN,NaN
tuplex-llvm16-hyper,2.500000,171.051636,0.020238,16.0,NaN,NaN,NaN,1.001151e+06,26745.0,15.522021,15.522021,0.030300,15.149666,0.033418,0.289921,NaN
tuplex-llvm16-hyper-noopt,2.500000,170.382859,0.020130,16.0,NaN,NaN,NaN,1.001122e+06,26745.0,15.460985,15.460985,0.029864,15.154437,0.035110,0.223353,NaN
tuplex-llvm16-nohyper,2.500000,182.090354,0.020099,16.0,NaN,NaN,NaN,1.001151e+06,26745.0,16.510268,16.510268,0.024510,14.186796,2.266430,NaN,NaN
tuplex-llvm9-hyper,2.500000,170.593243,0.020285,9.0,NaN,NaN,NaN,1.001122e+06,26745.0,15.480236,15.480236,0.027744,15.116777,0.033231,0.283469,NaN
tuplex-llvm9-hyper-noopt,2.500000,169.948438,0.020926,9.0,NaN,NaN,NaN,1.001151e+06,26745.0,15.421075,15.421075,0.027826,15.121187,0.030686,0.221773,NaN
